<a href="https://colab.research.google.com/github/adi-devv/Shell.ai-Hackathon/blob/main/Fuel_Blending_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
zip_path = '/content/drive/MyDrive/Colab_Projects/Shell_Hackathon/dataset.zip'
extract_path = '/content/colab_dataset_unzipped/'

!mkdir -p {extract_path}

!unzip -o {zip_path} -d {extract_path}

print("Dataset unzipped successfully!")

!ls {extract_path}
!ls {extract_path}/dataset/

Archive:  /content/drive/MyDrive/Colab_Projects/Shell_Hackathon/dataset.zip
   creating: /content/colab_dataset_unzipped/dataset/
  inflating: /content/colab_dataset_unzipped/dataset/sample_solution.csv  
  inflating: /content/colab_dataset_unzipped/dataset/test.csv  
  inflating: /content/colab_dataset_unzipped/dataset/train.csv  
Dataset unzipped successfully!
dataset
sample_solution.csv  test.csv  train.csv


In [3]:
data_directory = '/content/colab_dataset_unzipped/dataset/' # THIS IS THE CRUCIAL CHANGE

try:
    train_df = pd.read_csv(data_directory + 'train.csv')
    test_df = pd.read_csv(data_directory + 'test.csv')
    sample_submission_df = pd.read_csv(data_directory + 'sample_solution.csv')
except FileNotFoundError:
    print(f"Error: Make sure 'train.csv', 'test.csv', and 'sample_solution.csv' are in the directory: {data_directory}")
    exit()

print("Train data shape:", train_df.shape)
print("Test data shape:", test_df.shape)

Train data shape: (2000, 65)
Test data shape: (500, 56)


In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import mean_absolute_percentage_error, make_scorer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler # For potential future use
import warnings
import random # For setting multiple seeds

# Suppress potential warnings for cleaner output
warnings.filterwarnings('ignore')

# --- 1. Load Data ---
# Adjusted path based on your successful unzipping output
data_directory = '/content/colab_dataset_unzipped/dataset/' # THIS IS THE CRUCIAL CHANGE

try:
    train_df = pd.read_csv(data_directory + 'train.csv')
    test_df = pd.read_csv(data_directory + 'test.csv')
    sample_submission_df = pd.read_csv(data_directory + 'sample_solution.csv')
except FileNotFoundError as e: # Catch the specific error to print it
    print(f"Error loading data: {e}")
    print(f"Please ensure 'train.csv', 'test.csv', and 'sample_solution.csv' are in the directory: {data_directory}")
    raise # Re-raise the exception to stop execution and show the full traceback


print("Train data shape:", train_df.shape)
print("Test data shape:", test_df.shape)

# --- 2. Define Features and Targets ---
target_columns = [f'BlendProperty{i}' for i in range(1, 11)]
original_feature_columns = [col for col in train_df.columns if col not in ['ID'] + target_columns]

test_ids = test_df['ID']

print("\nOriginal Features for training shape (before FE):", train_df[original_feature_columns].shape)
print("Targets for training shape:", train_df[target_columns].shape)
print("Original Features for testing shape (before FE):", test_df[original_feature_columns].shape)


# --- 3. Enhanced Feature Engineering ---

def create_features(df, original_feats):
    df_processed = df.drop(columns=['ID'], errors='ignore').copy()

    # Start with original features that are relevant (not targets)
    df_features = df_processed[original_feats].copy()

    blend_cols = [f'Component{i}_vol' for i in range(1, 6)]

    # Ensure all blend_cols are float type to avoid issues with division
    for col in blend_cols:
        if col in df_features.columns:
            df_features[col] = df_features[col].astype(float)

    # Dictionary to hold all component property columns for easier access
    all_prop_cols = [f'Component{c}_Property{p}' for c in range(1, 6) for p in range(1, 11)]

    # === CORE FEATURE ENGINEERING ===

    # 1. Weighted Averages of Component Properties (Weighted by Volume) - Already good
    for prop_num in range(1, 11):
        weighted_avg_col = f'BlendProperty{prop_num}_WeightedAvg'
        df_features[weighted_avg_col] = 0.0
        for comp_num in range(1, 6):
            vol_col = f'Component{comp_num}_vol'
            prop_col = f'Component{comp_num}_Property{prop_num}'
            if vol_col in df_processed.columns and prop_col in df_processed.columns:
                # Add check for division by zero if vol_col sum isn't 100 always (though it should be)
                df_features[weighted_avg_col] += (df_processed[vol_col] / 100.0) * df_processed[prop_col]


    # 2. Interactions between Blend Volume and Component Properties - Already good
    for comp_num in range(1, 6):
        vol_col = f'Component{comp_num}_vol'
        for prop_num in range(1, 11):
            prop_col = f'Component{comp_num}_Property{prop_num}'
            if vol_col in df_processed.columns and prop_col in df_processed.columns:
                df_features[f'{vol_col}_x_{prop_col}'] = df_processed[vol_col] * df_processed[prop_col]

    # 3. Statistical Aggregations Across Component Properties - Expanded
    for prop_num in range(1, 11):
        props_for_agg = [f'Component{comp_num}_Property{prop_num}' for comp_num in range(1, 6)]
        existing_props_for_agg = [p for p in props_for_agg if p in df_processed.columns]

        if existing_props_for_agg:
            df_features[f'Property{prop_num}_min'] = df_processed[existing_props_for_agg].min(axis=1)
            df_features[f'Property{prop_num}_max'] = df_processed[existing_props_for_agg].max(axis=1)
            df_features[f'Property{prop_num}_mean'] = df_processed[existing_props_for_agg].mean(axis=1)
            df_features[f'Property{prop_num}_std'] = df_processed[existing_props_for_agg].std(axis=1).fillna(0)
            df_features[f'Property{prop_num}_range'] = df_features[f'Property{prop_num}_max'] - df_features[f'Property{prop_num}_min']

            # Count of non-zero components for this property (proxy for active ingredients)
            # This indicates how many components contribute to a specific property for a given blend.
            df_features[f'Property{prop_num}_active_components_count'] = (df_processed[existing_props_for_agg] != 0).sum(axis=1)


    # 4. More Complex Interactions (Interactions between properties of DIFFERENT components)
    # This can capture synergy or antagonism between ingredients.
    for i in range(1, 6): # Component i
        for j in range(i + 1, 6): # Component j (to avoid duplicates and self-interaction)
            for p1 in range(1, 11): # Property p1 of Component i
                for p2 in range(1, 11): # Property p2 of Component j
                    prop_i = f'Component{i}_Property{p1}'
                    prop_j = f'Component{j}_Property{p2}'
                    if prop_i in df_processed.columns and prop_j in df_processed.columns:
                        df_features[f'{prop_i}_x_{prop_j}'] = df_processed[prop_i] * df_processed[prop_j]
                        # Add a difference too if deemed relevant for specific properties
                        # df_features[f'{prop_i}_minus_{prop_j}'] = df_processed[prop_i] - df_processed[prop_j]


    # 5. Ratios (selective ratios if domain knowledge exists, or general ones)
    # Be careful with division by zero. Add a small epsilon if denominators can be zero.
    epsilon = 1e-6 # A tiny value to prevent division by zero

    # Example: Ratio of property 1 between component 1 and component 2
    for prop_num in range(1, 11):
        comp1_prop = f'Component1_Property{prop_num}'
        comp2_prop = f'Component2_Property{prop_num}'
        if comp1_prop in df_processed.columns and comp2_prop in df_processed.columns:
            df_features[f'Comp1_Prop{prop_num}_div_Comp2_Prop{prop_num}'] = \
                df_processed[comp1_prop] / (df_processed[comp2_prop] + epsilon)
            df_features[f'Comp2_Prop{prop_num}_div_Comp1_Prop{prop_num}'] = \
                df_processed[comp2_prop] / (df_processed[comp1_prop] + epsilon)

    # 6. Polynomial Features (simple, on volumes or key properties)
    # Example: Squares of blend volumes
    for vol_col in blend_cols:
        if vol_col in df_processed.columns:
            df_features[f'{vol_col}_sq'] = df_processed[vol_col] ** 2

    # Fill any remaining NaNs created during feature engineering (e.g., from std of single-item groups)
    df_features = df_features.fillna(0) # Or consider mean/median imputation

    return df_features


print("\nPerforming Feature Engineering...")
X_train_fe = create_features(train_df, original_feature_columns)
X_test_fe = create_features(test_df, original_feature_columns)

# Align columns after feature engineering to ensure consistency
train_cols = X_train_fe.columns
test_cols = X_test_fe.columns

missing_in_test = set(train_cols) - set(test_cols)
for c in missing_in_test:
    X_test_fe[c] = 0

missing_in_train = set(test_cols) - set(train_cols)
for c in missing_in_train:
    X_train_fe[c] = 0

X_test_fe = X_test_fe[train_cols] # Ensure column order is identical


y_train = train_df[target_columns]

print("Features after engineering (X_train_fe) shape:", X_train_fe.shape)
print("Features after engineering (X_test_fe) shape:", X_test_fe.shape)

# --- Define KFold here ---
kf = KFold(n_splits=5, shuffle=True, random_state=42) # 5-Fold Cross-Validation

# --- 4. Model Training (LightGBM with K-Fold Cross-Validation and Hyperparameter Tuning Placeholder) ---

# Define a custom MAPE scorer for RandomizedSearchCV
def custom_mape_scorer(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    if y_true.ndim == 1:
        return -mean_absolute_percentage_error(y_true, y_pred)
    else:
        return -np.mean([mean_absolute_percentage_error(y_true[:,i], y_pred[:,i]) for i in range(y_true.shape[1])])

mape_scorer = make_scorer(custom_mape_scorer, greater_is_better=True)


# Base LightGBM regressor for MultiOutputRegressor
lgbm = lgb.LGBMRegressor(objective='regression_l1', metric='mae', n_jobs=-1, random_state=42, verbose=-1, device='gpu') # Added device='gpu'

# Hyperparameter search space for RandomizedSearchCV
param_dist = {
    'estimator__n_estimators': [1000, 1500, 2000, 2500],
    'estimator__learning_rate': [0.005, 0.01, 0.02, 0.03],
    'estimator__num_leaves': [31, 64, 128],
    'estimator__max_depth': [-1, 10, 15],
    'estimator__feature_fraction': [0.7, 0.8, 0.9],
    'estimator__bagging_fraction': [0.7, 0.8, 0.9],
    'estimator__bagging_freq': [1],
    'estimator__lambda_l1': [0, 0.1, 0.5, 1],
    'estimator__lambda_l2': [0, 0.1, 0.5, 1],
    'estimator__min_child_samples': [20, 30, 40]
}

# Wrap LGBM in MultiOutputRegressor for tuning
multi_output_model_for_tuning = MultiOutputRegressor(lgbm)

print("\nStarting Hyperparameter Tuning with RandomizedSearchCV (this may take time)...")
random_search = RandomizedSearchCV(
    estimator=multi_output_model_for_tuning,
    param_distributions=param_dist,
    n_iter=20,
    scoring=mape_scorer,
    cv=kf,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train_fe, y_train)

best_lgbm_params = random_search.best_params_
print(f"\nBest Hyperparameters found: {best_lgbm_params}")

final_lgbm_params = {k.replace('estimator__', ''): v for k, v in best_lgbm_params.items()}

# Set up the final model with the best parameters, ensuring GPU is still specified
model = MultiOutputRegressor(lgb.LGBMRegressor(**final_lgbm_params, n_jobs=-1, random_state=42, verbose=-1, device='gpu')) # Added device='gpu'

# --- 5. Final Model Training and Prediction with K-Fold Cross-Validation ---

print("\nStarting Final K-Fold Cross-Validation training with best parameters...")
oof_preds = np.zeros(y_train.shape)
test_preds_folds = []
fold_mape_scores = []
num_seeds = 3
seeds = [random.randint(1, 1000) for _ in range(num_seeds)]

final_test_predictions_ensemble = []

for seed in seeds:
    print(f"\n--- Running K-Fold with seed: {seed} ---")
    current_seed_test_preds = []

    # Update model with current seed, ensuring GPU is specified
    model = MultiOutputRegressor(lgb.LGBMRegressor(**final_lgbm_params, n_jobs=-1, random_state=seed, verbose=-1, device='gpu')) # Added device='gpu'

    for fold, (train_index, val_index) in enumerate(kf.split(X_train_fe, y_train)):
        print(f"  --- Fold {fold+1}/{kf.n_splits} ---")
        X_train_fold, X_val_fold = X_train_fe.iloc[train_index], X_train_fe.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        model.fit(X_train_fold, y_train_fold)

        fold_val_preds = model.predict(X_val_fold)
        oof_preds[val_index] = fold_val_preds

        fold_test_preds = model.predict(X_test_fe)
        current_seed_test_preds.append(fold_test_preds)

        fold_mape = []
        for i, target_col in enumerate(target_columns):
            mape_val = mean_absolute_percentage_error(y_val_fold[target_col], fold_val_preds[:, i])
            fold_mape.append(mape_val)
        avg_fold_mape = np.mean(fold_mape)
        if seed == seeds[0]:
             fold_mape_scores.append(avg_fold_mape)
        print(f"  Fold {fold+1} Average MAPE (Seed {seed}): {avg_fold_mape:.4f}")

    final_test_predictions_ensemble.append(np.mean(current_seed_test_preds, axis=0))

print("\nFinal Cross-Validation Complete.")
overall_cv_mape = np.mean(fold_mape_scores)
print(f"Overall Cross-Validation Average MAPE (from first seed): {overall_cv_mape:.4f}")


# --- 6. Final Predictions and Submission ---

final_submission_predictions = np.mean(final_test_predictions_ensemble, axis=0)

predictions_df = pd.DataFrame(final_submission_predictions, columns=target_columns)

# --- 7. Generate Submission File ---
submission_df = pd.DataFrame({'ID': test_ids})
submission_df = pd.concat([submission_df, predictions_df], axis=1)

submission_df = submission_df[['ID'] + target_columns]

submission_file_name = 'submission.csv'
submission_df.to_csv(submission_file_name, index=False)

print(f"\nSubmission file '{submission_file_name}' created successfully!")
print("First 5 rows of the submission file:")
print(submission_df.head())

reference_cost_public = 2.72
estimated_score_public = 100 * max(0, 1 - overall_cv_mape / reference_cost_public)
print(f"\nEstimated Public Leaderboard Score (based on first seed's CV MAPE): {estimated_score_public:.2f}")

Train data shape: (2000, 65)
Test data shape: (500, 56)

Original Features for training shape (before FE): (2000, 55)
Targets for training shape: (2000, 10)
Original Features for testing shape (before FE): (500, 55)

Performing Feature Engineering...
Features after engineering (X_train_fe) shape: (2000, 1145)
Features after engineering (X_test_fe) shape: (500, 1145)

Starting Hyperparameter Tuning with RandomizedSearchCV (this may take time)...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
